In [ ]:
import telebot
import json
import feedparser
import pymorphy2
import re
import random
import time
import nltk
from nltk.corpus import stopwords


token = '980878523:AAEUYxMijMvEJTuS55crcbQc10JLbF_ApM4'
bot = telebot.TeleBot(token)


wait_for_category = False
wait_for_keywords = False
del_categories = False
del_keywords = False
last_joke = ''


# стартовая клавиатура
start_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
start_kbrd.row('🗣 Новости', '🤪 Анекдот')
start_kbrd.row('📟 Мой профиль')

# изменить преференции
pref_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
pref_kbrd.row('+ категория', '- категория')
pref_kbrd.row('+ ключ. слова', '- ключ. слова')
pref_kbrd.row('отмена')

# клавиатура с выбором категорий
categories_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
categories_kbrd.row('политика', 'спорт')
categories_kbrd.row('наука', 'искусство')
categories_kbrd.row('отмена')

# клавиатура общего назначения (отмены)
cancel_kbrd = telebot.types.ReplyKeyboardMarkup(True, True)
cancel_kbrd.row('отмена')

# приветствие
start_msg = '''
🤙 привет, любитель новостей!
давай я немного введу тебя в курс дела, вот мой скромный функционал:

🔹 подборка новостей, отобранных по твом категориям и ключевым словам (если таковые имеются)
🔹 конфигурация твоего профиля (добавление/удаление категорий, ключевых слов)
🔹 функция 'хочу смеяться 5 минут', она же - выдача случайного анекдота
🔹 функция 'добавить новый ресурс' (работает так: /newsource [link] [category]
'''


# стартовое сообщение
@bot.message_handler(commands=['start', 'help'])
def start_message(message):
    global start_kbrd
    global wait_for_category, wait_for_keywords, del_categories, del_keywords

    bot.send_message(message.chat.id, start_msg, 
                     reply_markup=start_kbrd)
    wait_for_category = False
    wait_for_keywords = False
    del_categories = False
    del_keywords = False
    
    # добавление нового пользователя в json-файл
    add_user(message.from_user.id)


@bot.message_handler(commands=['sources', 'links'])
def show_links(message):
    global start_kbrd
    with open('rss_bot_info.json', 'r') as file:
        info = json.load(file)['sources']
    links = []
    for category in info:
        links.extend(['<b>' + category + '</b>'] + info[category])
    
    bot.send_message(message.chat.id, 'Вот ссылки, которые у меня сейчас есть:\n' + '\n'.join(links), 
                     parse_mode='HTML', reply_markup=start_kbrd)
    
                     
# обработчик входящих сообщений
@bot.message_handler(content_types=['text'])
def send_message(message):
    global start_kbrd, pref_kbrd, categories_kbrd, cancel_kbrd
    global wait_for_category, wait_for_keywords, del_categories, del_keywords
    
    if message.text == '📟 Мой профиль':
        with open('rss_bot_info.json', 'r') as file:
            info = json.load(file)['users']
            _id = str(message.from_user.id)
            
            if _id in info.keys():
                bot.send_message(message.chat.id, f'ваш id:  {_id}' + f'\nкатегории:  {convert(info[_id]["categories"])}' +
                                f'\nключевые слова:  {convert(info[_id]["keywords"])}', reply_markup=pref_kbrd)
            else:
                add_user(message.from_user.id)
                bot.send_message(message.chat.id, f'ваш id:  {_id}' + f'\nкатегории:  {convert(info[_id]["categories"])}' +
                                f'\nключевые слова:  {convert(info[_id]["keywords"])}', reply_markup=pref_kbrd)
    
    elif message.text == '🗣 Новости':
        bot.send_message(message.chat.id, 'оставайтесь на линии.\nзапрос обрабатывается.')
        result = get_news(message.from_user.id)
        if result == 0:
            time.sleep(2)
            bot.send_message(message.chat.id, 'кажется, ты не выбрали никакую категорию.\nвыбери, пожалуйста.',
                            reply_markup=categories_kbrd)
            wait_for_category = True
        else:
            feed, info = result
            if feed == []:
                bot.send_message(message.chat.id, 'к сожалению, я ничего не нарыл(', reply_markup=start_kbrd)
            else:
                bot.send_message(message.chat.id, f'фух, я отсмотрел {info[1]} новостей за {round(info[0], 2)} сек.')
                for news in feed:
                    bot.send_message(message.chat.id, reformat(news), parse_mode='HTML')
                    time.sleep(5)
            bot.send_message(message.chat.id, 'ну как тебе? пойдет?', reply_markup=start_kbrd)
        
    elif message.text == '🤪 Анекдот':
        bot.send_message(message.chat.id, random_joke(), reply_markup=start_kbrd)
    
    elif message.text == '+ категория':
        wait_for_category = True
        bot.send_message(message.chat.id, 'выбери категорию.', reply_markup=categories_kbrd)
    
    elif message.text == '+ ключ. слова':
        wait_for_keywords = True
        bot.send_message(message.chat.id, 'введи ключевые слова через запятую.', reply_markup=cancel_kbrd)
    
    elif message.text == '- категория':
        del_categories = True
        bot.send_message(message.chat.id, 'от каких категорий ты хочешь отписаться?\nнапиши через запятую.')
    
    elif message.text == '- ключ. слова':
        del_keywords = True
        bot.send_message(message.chat.id, 'какие ключевые слова ты хочешь удалить?\nнапиши через запятую.')
    
    elif message.text == 'отмена':
        wait_for_category = False
        wait_for_keywords = False
        del_categories = False
        del_keywords = False
        bot.send_message(message.chat.id, 'что прикажешь делать дальше?', reply_markup=start_kbrd)
        
    elif message.text.startswith('/newsource'):
        ans = add_source(message.text[10:])
        if ans == 1:
            bot.send_message(message.chat.id, 'новая лента добавлена.')
        else:
            bot.send_message(message.chat.id, 'какие-то временные шоколодки с добавлением твоей ленты')
    else:
        
        if wait_for_category == True:
            add_preferences(message.from_user.id, 'categories', message.text)
            bot.send_message(message.chat.id, 'профиль обновлен.\nхочешь выбрать еще?', reply_markup=categories_kbrd)
            
        elif wait_for_keywords == True:
            add_preferences(message.from_user.id, 'keywords', message.text)
            bot.send_message(message.chat.id, 'профиль обновлен.', reply_markup=start_kbrd)
            wait_for_keywords = False
            
        elif del_categories == True:
            del_preferences(message.from_user.id, 'categories', message.text)
            bot.send_message(message.chat.id, 'профиль обновлен.', reply_markup=start_kbrd)
            del_categories = False
            
        elif del_keywords == True:
            del_preferences(message.from_user.id, 'keywords', message.text)
            bot.send_message(message.chat.id, 'профиль обновлен.', reply_markup=start_kbrd)
            del_keywords = False
        
        

# новый пользователь    
def add_user(user_id):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
        
    if str(user_id) not in list(current['users'].keys()):
        current['users'][str(user_id)] = {"categories": [], "keywords": []}
    
    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)    


# новые предпочтения пользователя (ключевые слова/категория)
def add_preferences(user_id, data_type, data):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    morph = pymorphy2.MorphAnalyzer()
    user_id = str(user_id)
    data = data.lower()
    
    if user_id in current['users'].keys():
        
        if data_type == 'categories':
            if data in current['sources'].keys():
                if data not in current['users'][user_id][data_type]:
                    current['users'][user_id][data_type].append(data)
        
        elif data_type == 'keywords':
            for word in data.replace(',','').split():
                if word not in current['users'][user_id][data_type]:
                    current['users'][user_id][data_type].append(word)
    else:
        add_user(user_id)
        add_preferences(user_id, data_type, data)
    
    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)        
        

# новый источник
def add_source(data):
    global bot
    link, category = data.lower().split()
    
    try:
        test_news = cleanhtml(feedparser.parse(link).entries[0].summary)
        if test_news == '':
            return 0
    except:
        return 0
        
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)    
        
    if category in current['sources'].keys():
        if link not in current['sources'][category]:
            current['sources'][category].append(link)
        else:
            return 0
    else:
        current['sources'][category] = [link]

    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)
    
    return 1


# удалить предпочтения
def del_preferences(user_id, data_type, data):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    user_id = str(user_id)
    data = data.lower().replace(',','').split()
    
    try:
        current['users'][user_id][data_type] = list(set(current['users'][user_id][data_type]) - set(data))
    except:
        return 'ошибка данных'
    
    with open('rss_bot_info.json', 'w') as write_obj:
        json.dump(current, write_obj, ensure_ascii=False)
    

# рандомный анекдот 
def random_joke():
    global last_joke
    
    link = 'http://www.jokesoftheday.net/jokes-feed/'
    jokes = feedparser.parse(link).entries
    ans = jokes[random.randint(0, len(jokes) - 1)].summary
    
    while len(ans.split()) > 50 and ans != last_joke:
        ans = jokes[random.randint(0, len(jokes) - 1)].summary
        ans = ans[:ans.index('Read more on') - 1]
    last_joke = ans
    
    return cleanhtml(last_joke)
    

# анализ новостей из пользовательских категорий по пользовательским ключевым словам
def get_news(user_id):
    with open('rss_bot_info.json', 'r') as read_obj:
        current = json.load(read_obj)
    
    start_time = time.time()
    chosen_news = []
    user_id = str(user_id)
    news_viewed = 0
    
    # запоминание пользовательских предпочтений
    try:
        categories = current['users'][user_id]['categories']
        keywords = current['users'][user_id]['keywords']
        links = [item for sublist in [current['sources'][x] for x in categories] for item in sublist]
    except:
        return -1
    
    if links:
    # анализ каждой новости из каждой ссылки
        for link in links:
            news_line = feedparser.parse(link).entries
            for news in news_line:

                news_viewed += 1
                content = cleanhtml(news.summary)
                # проверка валидности новости
                if check_it(content, keywords):

                    if 'document.write' in content:
                        content = content[: content.index('document.write') - 1]
                    clean = {'title': news.title, 'link': news.links[0]['href'], 'content': content}
                    chosen_news.append(clean)
                if len(chosen_news) == 7:
                    break
    else:
        return 0
                 
    return [chosen_news, [time.time() - start_time, news_viewed]]


# проверка новости на содержание ключевых слов
def check_it(text, keywords):
    morph = pymorphy2.MorphAnalyzer()
    
    if keywords:
        keywords = set([morph.parse(word.lower())[0].normal_form for word in keywords])
        word_list = set([morph.parse(i.lower())[0].normal_form for i in nltk.word_tokenize(text) \
                     if i not in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»«–' and i not in stopwords.words('russian')])
    
        return keywords & word_list != set()
    
    else:
        return True

    
# очистка самой новости от тэгов и прочего мусора
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return ' '.join(cleantext.split())    


# форматирование словаря - новости
def reformat(text):
    return '<b>' + text['title'] + '</b>' + '\n\n' + text['content'] + '\n\n' + text['link']


# маленькая вспомогательная функция
def convert(data):
    if not data:
        return '-'
    else:
        return ', '.join(data)

bot.polling(none_stop=True)

2019-11-13 21:37:08,632 (util.py:65 PollingThread) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}\']',)
Traceback (most recent call last):
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\util.py", line 59, in run
    task(*args, **kwargs)
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\__init__.py", line 276, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeout=timeout)
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\__init__.py", line 246, in get_updates
    json_updates = apihelper.get_updates(self.token, offset, limit, timeout, allowed_updates)
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\apihelper.py", line 180, in get_updates
    return _make_req

2019-11-13 21:37:27,994 (__init__.py:417 MainThread) ERROR - TeleBot: "A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:
[b'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}']"
2019-11-13 21:37:37,365 (util.py:65 PollingThread) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 409 Conflict. Response body:\n[b\'{"ok":false,"error_code":409,"description":"Conflict: terminated by other getUpdates request; make sure that only one bot instance is running"}\']',)
Traceback (most recent call last):
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\util.py", line 59, in run
    task(*args, **kwargs)
  File "C:\Users\busya\Anaconda3\lib\site-packages\telebot\__init__.py", line 276, in __retrieve_updates
    updates = self.get_updates(offset=(self.last_update_id + 1), timeo

In [76]:
import telebot

token = '980878523:AAEUYxMijMvEJTuS55crcbQc10JLbF_ApM4'
bot = telebot.TeleBot(token)

bot.send_message(268178395, '\n'.join(links), parse_mode='HTML')

In [5]:
test = 'A furniture read store keeps calling me All I wanted was one night stand Photo by Di_An_h on Unsplash Read more on page http://www.jokesoftheday.net'

In [9]:
test[:test.index('Read more on') - 1]

'A furniture read store keeps calling me All I wanted was one night stand Photo by Di_An_h on Unsplash'